# This is  a research notebook to try to find correlation between hyperwave and volume 

In [75]:
!pip install --upgrade pip
!pip install quandl

Requirement already up-to-date: pip in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (18.0)


In [76]:
from datetime import datetime
import json
import requests
import os.path as path

from scipy.spatial import ConvexHull

import quandl
quandl.ApiConfig.api_key = "cEofBzyzyihN3fj62kp4"

import pandas as pd
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='davzucky', api_key='aZw7LRJOSDcPJyIk2G0U')
# This is to avoid warning when dividing by zero
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'invalid': 'ignore', 'over': 'warn', 'under': 'ignore'}

In [77]:
path.abspath('.')

'/home/nbuser/library'

In [78]:
# mydata =  pd.DataFrame(quandl.get("FRED/GDP",returns="numpy", collapse="weekly",qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high'] },))
# mydata['diff'] = mydata['open'] - mydata['close']


In [79]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(mydata)

In [80]:
# f = mydata
# # display(f['open'])
# f = f.set_index('date')

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(weekly_data )

## This is the place where you setup the symbol and phases

In [81]:
data_3DSystemsCorp ={
    'name':'3D SYSTEMS CORP',
    'data_source':'quandl',
    'symbol':'DDD',
}


data_Valeant = {
    'name':'Baush health Company. Ex Valeant',
    'data_source':'quandl',
    'symbol':'VRX',
}

data_Amazon = {
    'name':'Amazon',
    'data_source':'quandl',
    'data_source':'investopedia',
    'symbol':'AMZN',
}

data_Netflix= {
    'name':'Netflix',
    'data_source':'investopedia',
    'symbol':'NFLX',
}

data_Apple = {
    'name':'Apple',
    'data_source':'investopedia',
    'symbol':'AAPL',
}

data_Intuit = {
    'name':'Intuit',
    'data_source':'investopedia',
    'symbol':'INTU',
}

data_Gilead_Sciences = {
    'name':'Gilead Sciences',
    'data_source':'investopedia',
    'symbol':'GILD',
}

data_Abiomed = {
    'name':'Abiomed',
    'data_source':'investopedia',
    'symbol':'ABMD',
}

data_Alphabet = {
    'name':'Alphabet',
    'data_source':'investopedia',
    'symbol':'GOOGL',
}

data_BTCUSD = {
    'name':'BTCUSD',
    'data_source':'CryptoCompare',
    'symbol':'BTC-USD',
}

data_DowJones = {
    'name':'Dow Jones',
    'data_source':'investopedia',
    'symbol':'^DJI',
}


data_DowJones_1920_1933 = {
    'name':'Dow Jones 1920-1933',
    'data_source':'LocalData',
    'symbol':'DowJones_1920_1933',
}



In [82]:
company_setup = data_DowJones_1920_1933 
name = company_setup ['name'] 
data_source = company_setup ['data_source'] 
symbol = company_setup ['symbol']

# constant used for other computation
root_date = datetime(1800, 1, 1)

# Helper function 
This section contain helper function that are here to load and clean the raw data

In [103]:
def add_weekid_and_price_is_closing_up(df):
#     df.loc[:,('date')] = pd.to_datetime(df.loc[:,('date')])
#     df = df.rename(columns={'Adj. Close':'close', 'Low':'low', 'Open':'open', 'High':'high',})
    df['is_price_closing_up'] = df.close > df.close.shift()
    df['weekId'] = df.apply (lambda row: get_nb_weeks (row),axis=1)
    df = df.drop("volume", axis=1)
    return df.sort_values(by="date")

class Investopedia_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe
    
    def _clean_data(self, df):
        df.loc[:,('date')] = pd.to_datetime(df.loc[:,('Date')])
        df = df.rename(columns={'Adj. Close':'close', 'Low':'low', 'Open':'open', 'High':'high',})
        df = df.set_index('date')
        df['date'] = df.index
        return df

    def _fetch_data(self): 
        url_symbol = "https://www.investopedia.com/markets/api/partial/historical/?Symbol={}&Type=Historical+Prices&Timeframe={}&StartDate=Jan+01%2C+1900".format(self._symbol, self._timeframe)
        df_list = pd.read_html(url_symbol, header=0, parse_dates=True)
        df_price = df_list[0].dropna()
        return df_price
    
    def get_dataframe(self):
        raw_data = self._fetch_data()
        return self._clean_data(raw_data)

class Quandl_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe

    def _aggregate_ticker_weekly(self, df):    
        open = df.open.resample('W-MON').last()
        close = df.close.resample('W-FRI').last().resample('W-MON').last()
        high = df.high.resample('W-MON').max()
        low = df.low.resample('W-MON').min()
        vol = df.volume.resample('W-MON').sum() 
        
        weekly_data = pd.concat([open, close, high, low, vol], axis=1)
        weekly_data ['date'] = weekly_data .index
        return weekly_data

    def _fetch_daily_data(self):
        daily_tickers = quandl.get_table('WIKI/PRICES', \
                                         ticker = [self._symbol], \
                                         qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high', 'volume'] }, \
                                         date = { 'gte': '1900-01-01'}, \
                                         paginate=True) #, 'lte': '2016-12-31' 
        daily_tickers = daily_tickers.set_index('date')
        daily_tickers['date'] = daily_tickers.index
        return daily_tickers
    
    def get_dataframe(self):
        daily_dataframe = self._fetch_daily_data()
        if( self._timeframe == 'daily'):
            return daily_dataframe.dropna()
        
        return self._aggregate_ticker_weekly(daily_dataframe).dropna()

class CryptoCompare_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe

    def _aggregate_ticker_weekly(self, df):    
        open = df.open.resample('W-MON').last()
        close = df.close.resample('W-SUN').last().resample('W-MON').last()
        high = df.high.resample('W-MON').max()
        low = df.low.resample('W-MON').min()
        vol = df.volume.resample('W-MON').sum() 
        
        weekly_data = pd.concat([open, close, high, low, vol], axis=1)
        weekly_data ['date'] = weekly_data .index
        return weekly_data

    def _fetch_daily_data(self):
        from_symbol, to_symbol  = self._symbol.split('-')
        url = "https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&allData=true&aggregate=3&e=CCCAGG".format( \
                                                                                                                        from_symbol, \
                                                                                                                        to_symbol)

        r = requests.get(url)
        array = json.dumps(r.json())
        
        data = json.loads(array)
        daily_tickers = pd.DataFrame(data["Data"])
        daily_tickers['date'] = pd.to_datetime(daily_tickers['time'],unit='s')
        daily_tickers = daily_tickers.rename( columns={"volumeto": "volume"})
        daily_tickers = daily_tickers.set_index('date')
        daily_tickers['date'] = daily_tickers.index
        return daily_tickers
    
    def get_dataframe(self):
        daily_dataframe = self._fetch_daily_data()
        if( self._timeframe == 'daily'):
            return daily_dataframe.dropna()
        
        return self._aggregate_ticker_weekly(daily_dataframe).dropna()

class LocalData_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._file_name = "{}.csv".format(symbol)
        self._timeframe = timeframe

    
    def get_dataframe(self):
        file_path = path.join(path.abspath('.'), 'data', self._file_name)
        df = pd.read_csv(file_path, header=0, parse_dates=True)
        df = df.rename(columns={column: column.lower() for column in df.columns})
        df.loc[:,('date')] = pd.to_datetime(df.loc[:,('date')])
        df = df.set_index('date')
        df['date'] = df.index
        return df
        

# https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&allData=true&aggregate=3&e=CCCAGG
    
sources = {
    "investopedia": Investopedia_Loader,
    "quandl": Quandl_Loader,
    "CryptoCompare": CryptoCompare_Loader,
    "LocalData": LocalData_Loader
}

def get_historical_data(symbol, source, timeframe='weekly'):
    source_class = sources[source](symbol, timeframe)
    dir(source_class)
    return source_class.get_dataframe()
    

In [84]:
# url = "https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&allData=true&aggregate=3&e=CCCAGG"
# df = pd.read_json(url)
# df = pd.DataFrame(df.locations.values.tolist())['Data']
# display(df)

# import json
# import pandas as pd
# import requests
# r = requests.get(url)
# # print r.json() #
# array = json.dumps(r.json())
# # print(data["Data"])
# data = json.loads(array)
# df = pd.DataFrame(data["Data"])
# df['date'] = pd.to_datetime(df['time'],unit='s')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df)

## Function that allow to calculate the weekid from a start date

In [85]:
def get_nb_weeks(row, base_date = root_date):
    return int((row["date"]-base_date).days/7)

def get_weekId_max_price(df):
    return df.loc[ df.loc[:,('close')].argmax()]['weekId']


In [86]:
def get_weekId_cartesian_product(df):
    df_from = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_from", "close": "close_from"})
    df_to = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_to", "close": "close_to"})
    df_cartesian = df_from.assign(foo=1).merge(df_to.assign(foo=1)).drop('foo', 1)

    return df_cartesian [(df_cartesian .weekId_from < df_cartesian .weekId_to)]    

# From here we start fetching the data
Above was only about setting up some basic function

In [104]:
# df_daily_price_raw = get_raw_historical_data(symbol, 'Daily')
df_weekly_price_raw = get_historical_data(symbol, data_source, 'Weekly')

df_weekly_price = add_weekid_and_price_is_closing_up(df_weekly_price_raw)

max_price_weekId = get_weekId_max_price(df_weekly_price)
df_weekly_price_until_max = df_weekly_price [(df_weekly_price.weekId <= max_price_weekId)]
# weekId_close_from_to = get_weekId_cartesian_product(df_weekly_price_until_max)

In [118]:
print(max_price_weekId)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_weekly_price)

6765


,open,high,low,close,date,is_price_closing_up,weekId
date,,,,,,,
1920-01-02,105.18,108.76,105.18,108.76,1920-01-02,False,6261
1920-01-09,108.85,108.85,106.59,106.59,1920-01-09,False,6262
1920-01-16,104.22,104.53,101.94,101.94,1920-01-16,False,6263
1920-01-23,102.72,103.48,101.90,101.90,1920-01-23,False,6264
1920-01-30,103.74,104.21,103.60,104.21,1920-01-30,True,6265
1920-02-06,103.01,103.01,95.50,95.75,1920-02-06,False,6266
1920-02-13,95.73,95.73,90.66,92.66,1920-02-13,False,6267
1920-02-20,92.60,95.57,92.60,95.57,1920-02-20,True,6268
1920-02-27,92.98,92.98,89.98,91.18,1920-02-27,False,6269


In [108]:
# print(max_price_weekId)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_weekly_price)
# df_weekly_price.ix[:df_weekly_price_until_max]

In [109]:
def get_line_slope_and_origine(df, x1_col_name = 'x1', \
                               y1_col_name = 'y1', \
                               x2_col_name = 'x2', \
                               y2_col_name = 'y2', \
                               m_col_name = 'm', \
                               b_col_name = 'b'):
    df[m_col_name] = (df[y1_col_name] - df[y2_col_name]) / (df[x1_col_name] - df[x2_col_name])
    df[b_col_name] = df[y1_col_name] - (df[x1_col_name] * df[m_col_name])
    return df

In [128]:
def get_mean_square_error(y_true, y_pred):
    y_square_diff = np.square(y_true-y_pred)
    return np.sum(y_square_diff) / len(y_true)

def nb_is_lower(y_true, y_pred):
    lower_item = y_true[y_true <= y_pred]
    return len(lower_item )
    
def get_y(x, m, b):
    return x * m + b

def calculate_mean_square_error(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return get_mean_square_error(df['close'], y_pred)

def nb_cut_price_low(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return nb_is_lower(df['low'], y_pred)

def get_hull_tangent(df_input):
    hull = ConvexHull(df_input[['weekId', 'close']].dropna())
    
    hull_results = [ [min(pair[0], pair[1]), max(pair[0], pair[1])] for pair in hull.simplices]
    data_from_to = [{"x1": df_input['weekId'].iloc[pair[0]], \
       "y1": df_input['close'].iloc[pair[0]], \
       "x2": df_input['weekId'].iloc[pair[1]], \
       "y2": df_input['close'].iloc[pair[1]]} for pair in hull_results]
    df = pd.DataFrame(data_from_to)
    df = get_line_slope_and_origine(df)
    df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
    df['days'] = np.abs(df['x1'] - df['x2'])
    df['mean_square_error'] = df.apply(lambda row: calculate_mean_square_error (row, df_input),axis=1)
    df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df_input),axis=1)
    df['ratio_error_cut'] = df['mean_square_error'] / df['nb_is_lower']
    df['ratio_slope_y1'] = df['m']/df['y1']
    df['ratio_slope_y2'] = df['m']/df['y2']
    return df

def get_upper_path(df):
    row = df.iloc[0]
    upper_path = [row['x1'], row['x2']]

    while(row['x2'] != max_price_weekId):
        row = df.loc[df['x1'] == row['x2']].head().iloc[0]
        upper_path.append(row['x2'])

    return upper_path

def delete_path(df, path):
    df_cleaned = df.copy(True)
    
    for x1, x2 in zip(path[:-1], path[1:]):
        df_cleaned = df_cleaned[(np.logical_not((df_cleaned.x1 == x1) & (df_cleaned.x2 == x2))) ]

    return df_cleaned.sort_values(['x1','y2'], ascending=[True, False])

def delete_upper_path(df):
    return delete_path(df, get_upper_path(df.sort_values(['x1','x2'], ascending=[True, False])))

def delete_shorter_path(df):
    df = df.sort_values(['x1','x2'], ascending=[True, False])
    path1 = get_upper_path(df)
    path2 = get_upper_path(df.iloc[1:])
    path = path1 if len(path1) < len(path2) else path2
    return delete_path(df, path)
    

In [111]:
# hull_results = [ [min(pair[0], pair[1]), max(pair[0], pair[1])] for pair in hull.simplices]
# data_from_to = [{"x1": df_weekly_price_until_max['weekId'].iloc[pair[0]], \
#    "y1": df_weekly_price_until_max['close'].iloc[pair[0]], \
#    "x2": df_weekly_price_until_max['weekId'].iloc[pair[1]], \
#    "y2": df_weekly_price_until_max['close'].iloc[pair[1]]} for pair in hull_results]
# df = pd.DataFrame(data_from_to)

In [112]:
df_hull_tangent = get_hull_tangent(df_weekly_price_until_max)
display(df_hull_tangent.sort_values(['x1','x2'], ascending=[True, False]))
py.iplot(graph_hyperwave(df_weekly_price_until_max,df_hull_tangent), filename='hyperwave')
# df_hull_tangent = delete_upper_path(df_hull_tangent)

,x1,x2,y1,y2,m,b,angle,days,mean_square_error,nb_is_lower,ratio_error_cut,ratio_slope_y1,ratio_slope_y2
0,6261,6765,108.76,380.30,0.538770,-3264.477976,28.314448,504,1.230070e+04,505,2.435782e+01,0.004954,0.001417
8,6261,6263,108.76,101.94,-3.410000,21458.770000,-73.655954,2,1.105660e+06,1,1.105660e+06,-0.031353,-0.033451
9,6263,6267,101.94,92.66,-2.320000,14632.100000,-66.682292,4,5.424419e+05,3,1.808140e+05,-0.022758,-0.025038
11,6267,6269,92.66,91.18,-0.740000,4730.240000,-36.501441,2,8.221629e+04,2,4.110815e+04,-0.007986,-0.008116
10,6269,6312,91.18,68.91,-0.517907,3337.938837,-27.379954,43,5.040669e+04,4,1.260167e+04,-0.005680,-0.007516
1,6312,6346,68.91,65.34,-0.105000,731.670000,-5.994093,34,1.542748e+04,5,3.085497e+03,-0.001524,-0.001607
2,6346,6492,65.34,89.18,0.163288,-970.883562,9.273850,146,4.758285e+03,3,1.586095e+03,0.002499,0.001831
7,6492,6630,89.18,153.86,0.468696,-2953.592174,25.112282,138,2.937055e+03,3,9.790185e+02,0.005256,0.003046
13,6630,6652,153.86,168.06,0.645455,-4125.503636,32.840404,22,6.171110e+03,5,1.234222e+03,0.004195,0.003841
12,6652,6703,168.06,203.30,0.690980,-4428.341569,34.643713,51,7.493923e+03,7,1.070560e+03,0.004112,0.003399


In [113]:
df_hull_tangent.sort_values(['x1','x2'], ascending=[True, False])

,x1,x2,y1,y2,m,b,angle,days,mean_square_error,nb_is_lower,ratio_error_cut,ratio_slope_y1,ratio_slope_y2
0,6261,6765,108.76,380.30,0.538770,-3264.477976,28.314448,504,1.230070e+04,505,2.435782e+01,0.004954,0.001417
8,6261,6263,108.76,101.94,-3.410000,21458.770000,-73.655954,2,1.105660e+06,1,1.105660e+06,-0.031353,-0.033451
9,6263,6267,101.94,92.66,-2.320000,14632.100000,-66.682292,4,5.424419e+05,3,1.808140e+05,-0.022758,-0.025038
11,6267,6269,92.66,91.18,-0.740000,4730.240000,-36.501441,2,8.221629e+04,2,4.110815e+04,-0.007986,-0.008116
10,6269,6312,91.18,68.91,-0.517907,3337.938837,-27.379954,43,5.040669e+04,4,1.260167e+04,-0.005680,-0.007516
1,6312,6346,68.91,65.34,-0.105000,731.670000,-5.994093,34,1.542748e+04,5,3.085497e+03,-0.001524,-0.001607
2,6346,6492,65.34,89.18,0.163288,-970.883562,9.273850,146,4.758285e+03,3,1.586095e+03,0.002499,0.001831
7,6492,6630,89.18,153.86,0.468696,-2953.592174,25.112282,138,2.937055e+03,3,9.790185e+02,0.005256,0.003046
13,6630,6652,153.86,168.06,0.645455,-4125.503636,32.840404,22,6.171110e+03,5,1.234222e+03,0.004195,0.003841
12,6652,6703,168.06,203.30,0.690980,-4428.341569,34.643713,51,7.493923e+03,7,1.070560e+03,0.004112,0.003399


In [114]:
print('Valeant : {}', 2.99/257.53)
print('netflix : {}', 7.065833/411.09)
print('amazon : {}', 17.596667/ 2012.71)

Valeant : {} 0.011610297829379104
netflix : {} 0.017188043980636843
amazon : {} 0.008742773176463573


In [115]:
def graph_hyperwave(df_data, df_hyperwave):

    x = df_data['weekId'].values
    y =  df_data['close'].values
    trace0 = go.Scatter(
        x = x,
        y = y,
        mode = 'lines',
        name = 'lines'
    )

    layout = {
    #     'xaxis': {
    #         'range': [0, 7]
    #     },
    #     'yaxis': {
    #         'range': [0, 2.5]
    #     },
        'shapes': [{
                'type': 'line',
                'x0': row[1]['x1'],
                'y0': row[1]['y1'],
                'x1': row[1]['x2'],
                'y1': row[1]['y2'],
                'line': {
                    'color': 'rgb(55, 128, 191)',
                    'width': 3,
                }
            } for row in df_hyperwave.iterrows()
            # Line Vertical
            ]
    }
    data = [trace0]

    # py.iplot(data, filename='line-mode')
    fig = {
        'data': data,
        'layout': layout,
    }

    return fig
#     py.iplot(fig, filename='shapes-lines')

In [116]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_weekly_price_until_max)
# df =delete_upper_path(df_weekly_price_until_max)
# display(df)

In [132]:
df = df_weekly_price_until_max
max_weekId = 6492
nb_tail_items = 104

def get_max_price(df, column_name = 'close'):
    return df.loc[ df.loc[:,(column_name)].argmax()][column_name]
# weekId_max_price = get_weekId_max_price(last_n_weeks_Items) 
#     max_price_last_n_weeks = last_n_weeks_Items[(last_n_weeks_Items.weekId == weekId_max_price )].iloc[0]

last_n_weeks_Items = df[(df.weekId < max_weekId)].tail(nb_tail_items)    
max_price = get_max_price(last_n_weeks_Items )

print( max_price)

df_week_greater_than = df[(df.weekId >= max_weekId)]

# df.loc[ df.loc[:,('close')].argmax()]['weekId']
df_val_price_greater_than_max = df_week_greater_than[(df_week_greater_than.close > max_price)]
weekId_start_hyperwave = df_val_price_greater_than_max.loc[ df_val_price_greater_than_max.loc[:,('weekId')].argmin()]['weekId']
df_week_greater_than = df[(df.weekId >= weekId_start_hyperwave )]
df_hyperwave = get_hull_tangent(df_week_greater_than).sort_values(['x1','y2'], ascending=[True, False])


df_hyperwave = delete_shorter_path(df_hyperwave)
display(df_hyperwave)
py.iplot(graph_hyperwave(df_weekly_price_until_max,df_hyperwave), filename='hyperwave')
# py.iplot(fig, filename='shapes-lines')

104.65


,x1,x2,y1,y2,m,b,angle,days,mean_square_error,nb_is_lower,ratio_error_cut,ratio_slope_y1,ratio_slope_y2
8,6514,6589,104.86,136.27,0.418800,-2623.203200,22.723935,75,2.689091e+03,5,537.818274,0.003994,0.003073
9,6589,6630,136.27,153.86,0.429024,-2690.571707,23.220513,41,2.611741e+03,6,435.290216,0.003148,0.002788
11,6630,6652,153.86,168.06,0.645455,-4125.503636,32.840404,22,1.842099e+03,5,368.419712,0.004195,0.003841
10,6652,6703,168.06,203.30,0.690980,-4428.341569,34.643713,51,1.807933e+03,7,258.276170,0.004112,0.003399
2,6703,6707,203.30,208.79,1.372500,-8996.567500,53.923015,4,8.036207e+03,5,1607.241383,0.006751,0.006574
3,6707,6752,208.79,299.10,2.006889,-13251.413778,63.513673,45,2.221160e+04,6,3701.933562,0.009612,0.006710
1,6752,6762,299.10,344.80,4.570000,-30557.540000,77.657174,10,2.407026e+05,6,40117.094326,0.015279,0.013254
0,6762,6765,344.80,380.30,11.833333,-79672.200000,85.169580,3,2.323923e+06,4,580980.838188,0.034319,0.031116


In [120]:

# display(df_hyperwave.sort_values(['x1','y2'], ascending=[True, False]))
# print(max_price_weekId)
# print(get_upper_path(df_hyperwave))
df_low_path = delete_upper_path(df_hull_tangent)
display(df_hull_tangent)
display(df_low_path )
print(df_low_path.std()) 
print(df_low_path.mean()) #(axis=['m'])
print(df_low_path.median())

,x1,x2,y1,y2,m,b,angle,days,mean_square_error,nb_is_lower,ratio_error_cut,ratio_slope_y1,ratio_slope_y2
0,6261,6765,108.76,380.30,0.538770,-3264.477976,28.314448,504,1.230070e+04,505,2.435782e+01,0.004954,0.001417
1,6312,6346,68.91,65.34,-0.105000,731.670000,-5.994093,34,1.542748e+04,5,3.085497e+03,-0.001524,-0.001607
2,6346,6492,65.34,89.18,0.163288,-970.883562,9.273850,146,4.758285e+03,3,1.586095e+03,0.002499,0.001831
3,6762,6765,344.80,380.30,11.833333,-79672.200000,85.169580,3,1.029323e+07,4,2.573306e+06,0.034319,0.031116
4,6752,6762,299.10,344.80,4.570000,-30557.540000,77.657174,10,1.240290e+06,6,2.067149e+05,0.015279,0.013254
5,6703,6707,203.30,208.79,1.372500,-8996.567500,53.923015,4,5.995693e+04,5,1.199139e+04,0.006751,0.006574
6,6707,6752,208.79,299.10,2.006889,-13251.413778,63.513673,45,1.582923e+05,6,2.638204e+04,0.009612,0.006710
7,6492,6630,89.18,153.86,0.468696,-2953.592174,25.112282,138,2.937055e+03,3,9.790185e+02,0.005256,0.003046
8,6261,6263,108.76,101.94,-3.410000,21458.770000,-73.655954,2,1.105660e+06,1,1.105660e+06,-0.031353,-0.033451
9,6263,6267,101.94,92.66,-2.320000,14632.100000,-66.682292,4,5.424419e+05,3,1.808140e+05,-0.022758,-0.025038


,x1,x2,y1,y2,m,b,angle,days,mean_square_error,nb_is_lower,ratio_error_cut,ratio_slope_y1,ratio_slope_y2
8,6261,6263,108.76,101.94,-3.410000,21458.770000,-73.655954,2,1.105660e+06,1,1.105660e+06,-0.031353,-0.033451
9,6263,6267,101.94,92.66,-2.320000,14632.100000,-66.682292,4,5.424419e+05,3,1.808140e+05,-0.022758,-0.025038
11,6267,6269,92.66,91.18,-0.740000,4730.240000,-36.501441,2,8.221629e+04,2,4.110815e+04,-0.007986,-0.008116
10,6269,6312,91.18,68.91,-0.517907,3337.938837,-27.379954,43,5.040669e+04,4,1.260167e+04,-0.005680,-0.007516
1,6312,6346,68.91,65.34,-0.105000,731.670000,-5.994093,34,1.542748e+04,5,3.085497e+03,-0.001524,-0.001607
2,6346,6492,65.34,89.18,0.163288,-970.883562,9.273850,146,4.758285e+03,3,1.586095e+03,0.002499,0.001831
7,6492,6630,89.18,153.86,0.468696,-2953.592174,25.112282,138,2.937055e+03,3,9.790185e+02,0.005256,0.003046
13,6630,6652,153.86,168.06,0.645455,-4125.503636,32.840404,22,6.171110e+03,5,1.234222e+03,0.004195,0.003841
12,6652,6703,168.06,203.30,0.690980,-4428.341569,34.643713,51,7.493923e+03,7,1.070560e+03,0.004112,0.003399
5,6703,6707,203.30,208.79,1.372500,-8996.567500,53.923015,4,5.995693e+04,5,1.199139e+04,0.006751,0.006574


x1                   2.112919e+02
x2                   2.113072e+02
y1                   8.890739e+01
y2                   1.074721e+02
m                    3.755816e+00
b                    2.503772e+04
angle                5.239082e+01
days                 4.923101e+01
mean_square_error    2.811885e+06
nb_is_lower          1.724633e+00
ratio_error_cut      7.408966e+05
ratio_slope_y1       1.634145e-02
ratio_slope_y2       1.617379e-02
dtype: float64
x1                   6.493538e+03
x2                   6.532308e+03
y1                   1.535292e+02
y2                   1.744169e+02
m                    1.127556e+00
b                   -7.697333e+03
angle                1.322461e+01
days                 3.876923e+01
mean_square_error    1.043791e+06
nb_is_lower          4.153846e+00
ratio_error_cut      3.205026e+05
ratio_slope_y1       9.785421e-04
ratio_slope_y2      -4.582648e-04
dtype: float64
x1                    6492.000000
x2                    6630.000000
y1                

In [ ]:
# def get_line_slope(row):
#     return (row.close_from - row.close_to) / (row.weekId_from - row.weekId_to)


# weekId_close_from_to['slope'] = weekId_close_from_to.apply (lambda row: get_line_slope(row),axis=1)



# weekId_close_from_to

# # m = -0.110000
# # b = 1092.930000
# # m = 15.82
# # b = -177675.62
# def is_phase_below(m, b):
# #     m = row.m
# #     b = row.b
#     df = df_weekly_price
#     df_weekly_close = df[["weekId", "close"]]
#     df_weekly_close['ln_y'] = (df_weekly_close.weekId * m) + b
#     df_weekly_close['is_below'] = df_weekly_close.ln_y.le(df_weekly_close.close) | np.isclose(df_weekly_close['ln_y'], df_weekly_close['close'])
#     return df_weekly_close['is_below'].all()


# weekId_close_from_to['is_below'] = weekId_close_from_to.apply (lambda row: is_phase_below(row, df_weekly_price),axis=1)
# is_phase_below()

# weekId_close_from_to['r'] = weekId_close_from_to(weekId_close_from_to['m'], weekId_close_from_to['b'] )

# weekId_close_from_to
# import numpy as np
# import pandas as pd
# np.seterr(divide='ignore', invalid='ignore')



# df = df_weekly_price_until_max[:15].rename(columns={'weekId':'x', 'close': 'y'})


# data = {'x':[1,2,3,4],'y':[5,6,10,12]}
# df = pd.DataFrame(data)



# calculate m and b from y = mx = b
# df_m = (df['y'].values - df['y'].values[:, None]) / (df['x'].values - df['x'].values[:, None])
# df_b = df['y'].values - (df['x'].values * df_m)


# import itertools
# nb_partition = int((df['x'].values.shape[0] / 10)) + 1
# x_values_split = np.array_split(df['x'].values, nb_partition) 
# y_values_split = np.array_split(df['y'].values, nb_partition)              


# def get_y_values(x_value, m, b):
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
#     x_cube = np.reshape(x_values, cube_shape)
    
#     return (x_cube * m) + b 

# def get_is_line_above_close(x_values, y_values, m, b):
#     y_lenght = y_values.shape[0]
#     cube_shape = (y_lenght, 1, 1)
    
#     y_cube = np.reshape(y_values, cube_shape)
#     return get_y_values(x_values,m, b) < y_cube

    
# result = [get_y_vlowalues(x_values, df_m, df_b) for (x_values, y_values) in itertools.zip_longest(x_values_split, y_values_split )]
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
    
#     x_cube = np.reshape(x_values, cube_shape)
#     y_cube = np.reshape(y_values, cube_shape)
#     cube_y_value_calculated =  (x_cube * df_m) + df_b 

# x_values = x_values_split[0]
# y_values = y_values_split[0]              
# x_lenght = x_values.shape[0]
# cube_shape = (x_lenght, 1, 1)

# x_cube = np.reshape(x_values, cube_shape)
# y_cube = np.reshape(y_values, cube_shape)
